## Обработка данных

In [1]:
import pandas as pd
import os

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

os.chdir('../')

In [2]:
df = pd.read_excel('data/orig/test.xlsx')
df.head()

,Id,text,answer
0,672527f94ac0e6c927fe9209,"Обратить\t взыскание\t на\t денежные\t средства\t должника\t КОВАЛЕВ\t ВЯЧЕСЛАВ\t ГЕННАДЬЕВИЧ\t 03.09.1978\nг.р.,\tнаходящиеся\tна\tсчетах\tв\tбанках\tи\tиных\tкредитных\tорганизациях:\nЮГО-ЗАПАДНЫЙ\tБАНК\tПАО\tСБЕРБАНК,\tБИК\t046015602,\tсчет:\t40817810630061819252;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601398000003700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601398500003700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601933800003700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601933300003700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601840100003700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601840600003700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601810800003700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601810700001700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601978200003700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601810300003700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601840000001700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601978700003700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601840500001700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601810200001700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601978600001700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601978100001700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601156200003700124;\nФИЛИАЛ\tNo\t7701\tБАНКА\tВТБ\t(ПАО),\tБИК\t044525745,\tсчет:\t30601810205052519251;\nФИЛИАЛ\tNo\t2351\tБАНКА\tВТБ\t(ПАО),\tБИК\t040349758,\tсчет:\t40817840932556001971;\nФИЛИАЛ\tNo\t2351\tБАНКА\tВТБ\t(ПАО),\tБИК\t040349758,\tсчет:\t40817810832556010898;\nЮГО-ЗАПАДНЫЙ\tБАНК\tПАО\tСБЕРБАНК,\tБИК\t046015602,\tсчет:\t40817810530062328130;\nФИЛИАЛ\tNo\t7701\tБАНКА\tВТБ\t(ПАО),\tБИК\t044525745,\tсчет:\t40817810722254002807;\nФИЛИАЛ\tNo\t7701\tБАНКА\tВТБ\t(ПАО),\tБИК\t044525745,\tсчет:\t40817978222254002168;\nФИЛИАЛ\tNo\t7701\tБАНКА\tВТБ\t(ПАО),\tБИК\t044525745,\tсчет:\t40817840622254002168;\nКБ\t""Ренессанс\tКредит""\t(ООО),\tБИК\t044525135,\tсчет:\t40817810997750205910;\nПАО\tСБЕРБАНК,\tБИК\t044525225,\tсчет:\t40817810938049607994;\nПАО\tСБЕРБАНК,\tБИК\t044525225,\tсчет:\t40817810038064928749;\nПАО\tСБЕРБАНК,\tБИК\t044525225,\tсчет:\t40817810038064927313;\nПАО\tСБЕРБАНК,\tБИК\t044525225,\tсчет:\t40817810138170573768;\nПАО\tСБЕРБАНК,\tБИК\t044525225,\tсчет:\t40817810540005904391;\nПАО\tРОСБАНК,\tБИК\t044525256,\tсчет:\t40817810097440007054;\nКУРСКОЕ\tОТДЕЛЕНИЕ\tN8596\tПАО\tСБЕРБАНК,\tБИК\t043807606,\tсчет:\t42307810033000116032;\nАО\t""АЛЬФА-БАНК"",\tБИК\t044525593,\tсчет:\t40817810506370028148;\nБАНК\tГПБ\t(АО),\tБИК\t044525823,\tсчет:\t40817810964071606339;\nБАНК\tГПБ\t(АО),\tБИК\t044525823,\tсчет:\t40817810364071611665;\nФИЛИАЛ\tЦЕНТРАЛЬНЫЙ\tПАО\tБАНКА\t""ФК\tОТКРЫТИЕ"",\tБИК\t044525297,\tсчет:\t40817810978006973068;\nФИЛИАЛ\tNo\t2351\tБАНКА\tВТБ\t(ПАО),\tБИК\t040349758,\tсчет:\t40817978432556001929;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601156700003700129;\nнезависимо\tот\tостатка\tденежных\tсредств\tна\tсчете.",NaN
1,672526134ac0e6c927fe3400,"Обратить\t взыскание\t на\t денежные\t средства\t должника\t Арбузова\t Михаила\t Михайловича\t 03.03.1976\t г.р.,\nнаходящиеся\tна\tсчетах\tв\tбанках\tи\tиных\tкредитных\tорганизациях:\nАО\t""ТИНЬКОФФ\tБАНК"",\tБИК\t044525974,\tсчет:\t40817810600001600017;\nОРЕНБУРГСКОЕ\tОТДЕЛЕНИЕ\tN8623\tПАО\tСБЕРБАНК,\tБИК\t045354601,\tсчет:\t42307810046140126000;\nОРЕНБУРГСКОЕ\tОТДЕЛЕНИЕ\tN8623\tПАО\tСБЕРБАНК,\tБИК\t045354601,\tсчет:\t42307810646007402351;\nОРЕНБУРГСКОЕ\tОТДЕЛЕНИЕ\tN8623\tПАО\tСБЕРБАНК,\tБИК\t045354601,\tсчет:\t42307810346005600057;\nОРЕНБУРГСКОЕ\tОТДЕЛЕНИЕ\tN8623\tПАО\tСБЕРБАНК,\tБИК\t045354601,\tсчет:\t40817810146007400345;\nОРЕНБУРГСКОЕ\tОТДЕЛЕНИЕ\tN8623\tПАО\tСБЕРБАНК,\tБИК\t045354601,\tсчет:\t42307840146007400030;\nОРЕНБУРГСКОЕ\tОТДЕЛЕНИЕ\tN8623\tПАО\tСБЕРБАНК

In [3]:
# Проверяем на пустые значения в столбцах, если вдруг надо обработать их
df.isnull().sum()

Id           0
text         0
answer    1000
dtype: int64

Так как цифры встречаются очень часто и не несут смысловой нагрузки, можем обработать их, как и знаки препинания

In [ ]:
def apply_and_check(dataframe, func, column, target_column=None, **kwargs):
    if target_column is None:
        target_column = column

    total_before = dataframe[column].str.len().sum()
    dataframe[target_column] = dataframe[column].apply(func, **kwargs)

    total_after = dataframe[target_column].str.len().sum()
    total_removed = total_before - total_after
    
    print(f'Символов до: {total_before:,}, '
            f'после: {total_after:,}')
    print(f'Удалено символов: {total_removed:,} ({total_removed/total_before*100:.2f}%)')

### Символы

In [5]:
import re

# Удаляем табуляции/переносы строк, цифры, знаки препинания и приводим к нижнему регистру
def clean_text(text):
    # text = re.sub(r"[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-\t\n№]+", ' ', text).lower()
    
    text = re.sub(r'[^а-яё\s]+', ' ', text.lower()) # Оставляем только кириллицу
    text = re.sub(r'\s+', ' ', text).strip()  # Убираем табуляцию/переносы и склеиваем все whitespace в один пробел
    return text

In [6]:
apply_and_check(df, clean_text, 'text', target_column='clean')

Символов до: 1,036,674, после: 893,443
Удалено символов: 143,231 (13.82%)


### Стоп-слова

In [7]:
from nltk.corpus import stopwords

stopwords_ru = stopwords.words("russian")

# Оставляем только "не" из всех отдельных стоп-слов для последующего соеденения после лемматизации
stopwords_ru.remove('не')

def remove_stopwords(text):
    # Если попадётся не слово или нуль
    if not isinstance(text, str) or not text.strip():
        return text
    
    words = text.split()
    
    filtered_words = [word for word in words 
                       if word and word not in stopwords_ru]
    
    return ' '.join(filtered_words)

In [8]:
apply_and_check(df, remove_stopwords, 'clean')

Символов до: 893,443, после: 835,537
Удалено символов: 57,906 (6.48%)


### Лишние слова по частоте

Не уверен, лучше **сначала** убрать лишние слова по частоте или проверить орфографию и сделать лемматизацию и **потом** только убрать лишние слова.  
В **первом случае** убираются все слова из списка ниже точно, но пропускаются такие же слова, если они с ошибками.  
А во **втором случае** проверка орфографии может изменить некоторые слова неправильно (что как раз и было замечено) и лемматизация может ошибочно сделать не начальную форму слов.

Также нужно убрать часто встречающиеся лишние слова по типу "счёт, банк" и подобные  
Выберем из часто встречающихся слов через value_counts(), которые могут не нести смысла для классификации

In [9]:
# Подсчет слов
word_counts = df['clean'].str.split().explode().value_counts()

# Вывод топ слов
word_counts.head(50)

clean
не                 3150
указано            1892
сведений           1661
банк               1563
запрос             1486
дата               1252
мвв                1074
должника            973
счета               770
россии              735
г                   713
исполнительного     661
актуальности        647
сумме               641
пао                 596
сведения            584
постановление       579
производства        567
счет                551
средств             545
сумма               518
связи               508
денежных            493
бик                 473
код                 468
удовлетворению      468
ип                  466
ао                  463
признает            452
валюта              450
заявителя           445
наличие             405
обстоятельства      400
заявлении           389
судебного           386
подразделения       383
рассмотрев          381
валюте              378
доводы              372
счете               365
единица             359
курса     

In [ ]:
# Выберем те, которые кажутся лишними
# Точно надо удалить их перед проверкой орфографии как можно больше, так как слишком долго обрабатывает,
# но и искать каждое слово в каждом падеже как-то неправильно. Может снова пройтись после лемматизации, так как это может убрать ещё слова
unnecessary_words = {
    'счет', 'счета', 'банк', 'банка', 'мвв', 'г', 'валюта', 'сумма',
    'сумме', 'рубль', 'курс', 'бик', 'код', 'адрес', 'россия', 'россии', 
    'филиал', 'отделение'
}

def remove_unnecessary_words(text):
    pattern = r'\b(' + '|'.join(re.escape(word) for word in unnecessary_words) + r')\b'
    return re.sub(pattern, '', text, flags=re.IGNORECASE)

In [11]:
apply_and_check(df, remove_unnecessary_words, 'clean')

Символов до: 835,537, после: 796,391
Удалено символов: 39,146 (4.69%)


В итоге после базовой очистки осталось ~80% символов всего датасета

### Отдельный столбец для методов выделения ключевых слов

Для этих методов нужно убрать цифры, но не пунктуацию и стоп-слова

In [12]:
# Отдельный класс под эти методы. Тут сложновато, так как даты, хар-ки машин, счета и другие создают множество пробелов и пустых пунктуаций
def clean_text_keyword(text, lower=True):
    # Приводим к нижнему регистру
    if lower:
        text = text.lower()

    # Убираем отдельно обозначения машин и подобные, так как они состоят не только из цифр
    text = re.sub(r'\bvin\s*[a-z0-9]{17}\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\b(?=.*\d)(?=.*[a-z])[a-z0-9]{6,}\b', '', text, flags=re.IGNORECASE)
    
    # Убираем ВСЕ цифры
    text = re.sub(r'\d+', '', text)
    
    # Сжимаем повторяющиеся одинаковые знаки пунктуации (.,!? и т.д.)
    # "????" -> "?", ",...." -> ",."
    text = re.sub(r'([.,!?;:—\-])\1+', r'\1', text)
    
    # Убираем множественные пробелы (включая пробелы вокруг пунктуации)
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Убираем пунктуацию, стоящую отдельно от слов
    text = re.sub(r'\s+([.,!?;:—\-])\s+', ' ', text)
    return text

In [13]:
df_keywords = df[['Id', 'text', 'answer']].copy()

apply_and_check(df_keywords, clean_text_keyword, 'text', target_column='clean_with_stopwords') 
print()

apply_and_check(df_keywords, clean_text_keyword, 'text', target_column='clean_with_stopwords_with_casing', lower=False)

# Можно через csv файл, без parquet, joblib, pickle так как датасет буквально 1000 строк
df_keywords.to_csv('data/processed/for_keywords.csv', index=False)

Символов до: 1,036,674, после: 938,817
Удалено символов: 97,857 (9.44%)

Символов до: 1,036,674, после: 938,817
Удалено символов: 97,857 (9.44%)


### Орфография

Используем Hunspell, то есть spylls, который то же самое, но новее.  
Может стоило использовать [JamSpell](https://github.com/bakwc/JamSpell), который намного быстрее и точнее, но это только на Linux, а для Windows,  версия JamSpellPro, просто не открывается сайт с инструкциями.  
Можно попробовать также C++ через обёртку Cynthon с помощью CyHunspell, но оно при установке выдаёт ошибку. Кажется оно не работает на python 3.12, только [3.10](https://pypi.org/project/cyhunspell-py310/).  
Ещё можно попробовать SymSpell, который будет поновее и быстрее во много раз со своими фичами.  
Так же попробуем PySpellChecker.

In [ ]:
from tqdm import tqdm
import time

def print_spell_check_stats(stats, elapsed_time):
    corrected_words_percent = stats['corrected_words'] / stats['total_words'] * 100
    speed = stats['total_words'] / elapsed_time
    
    print(f'Слов проверено: {stats["total_words"]:,}')
    print(f'Исправлено слов: {stats["corrected_words"]:,} ({corrected_words_percent:.2f})%, '
          f'уникальных слов: {len(stats["unique_words"]):,}')
    print(f'Попаданий в кэш: {stats["cached_hits"]:,}')
    print(f'Скорость: {speed:.0f} слов/сек')

#### Spylls

In [ ]:
%%script cmd /c ""
# Добавил там, где неудачные методы, чтобы пропускать ячейки, если буду нажимать "Run All Cells"

from spylls.hunspell import Dictionary

# Скачал словарик мозиллы [https://addons.mozilla.org/ru/firefox/addon/russian-spellchecking-dic-3703/]
dictionary = Dictionary.from_zip('dictionaries/russian_spellchecking_dic_3703-0.4.5.2resigned1.xpi')

word_cache = {}
stats = {
    'total_words': 0,
    'corrected_words': 0,
    'cached_hits': 0,
    'start_time': None,
}

def spell_check_spylls(text):
    # Вдруг попадётся не строка или нуль
    if not isinstance(text, str) or not text.strip():
        return text

    words = text.split()
    corrected_words = []
    
    for word in words:
        # Проверка на пустые слова или слова только из пробелов (можно убрать позже, если выявится, что слова не склеены случайно)
        if not word.strip():
            corrected_words.append(word)
            continue

        stats['total_words'] += 1
        
        # Проверяем кэш первым
        if word in word_cache:
            stats['cached_hits'] += 1
            corrected_words.append(word_cache[word])
            continue
        
        # Если нет в кэше - проверяем и кэшируем результат
        if dictionary.lookup(word):
            word_cache[word] = word
            corrected_words.append(word)
        else:
            suggestions = list(dictionary.suggest(word))
            corrected = suggestions[0] if suggestions else word

            # Если слово было исправлено
            if corrected != word:
                stats['corrected_words'] += 1
            
            word_cache[word] = corrected
            corrected_words.append(corrected)
    
    return ' '.join(corrected_words)

In [16]:
%%script cmd /c ""
stats['start_time'] = time.time()

tqdm.pandas(desc="Spylls")
df['corrected'] = df['clean'].progress_apply(spell_check_spylls) 
# Долго, первые 5 строк делает 6.7 секунды, без кэша ещё в 5x медленее.
# А мультипроцессорность и мультипоточность не помогает, скорее всего из-за GIL и то, что работаем в ipynb файле

elapsed_time = time.time() - stats['start_time']
print_spell_check_stats(stats, elapsed_time)

#### SymSpell

```python 
sym_spell.load_dictionary('dictionaries/ru-100k.txt', term_index=0, count_index=1)
```
Не хочет нормально открывать словарь и выдаёт слово "Рё" на каждое короткое слово, а другие не исправляет, перед этим надо исправить txt файл, так как изначально кодировка utf-8, а стандартная кодировка для моей операционной системы для python - windows-1251 и таким образом не получается прочитать файл в кодировке utf-8.  
Можно обойти через смену стандартной кодировки или пересозданием объекта словаря.

In [ ]:
from symspellpy import SymSpell, Verbosity

entries = []
with open('dictionaries/ru-100k.txt', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split()
        entries.append((parts[0].lower(), int(parts[1])))

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
for word, freq in entries:
    sym_spell.create_dictionary_entry(word, freq)

In [18]:
word_cache = {}
stats = {
    'total_words': 0,
    'corrected_words': 0,
    'cached_hits': 0,
    'start_time': None,
    'unique_words': set() # Можно через set, чтобы не дублировать слова
}
min_len = 3 # SymSpell пытается исправить множество мелких слов, даже если это не ошибка по типу ип, пао и подобное. Ограничим

def spell_check_symspell(text):
    if not isinstance(text, str) or not text.strip():
        return text
    
    words = text.split()
    corrected_words = []

    for word in words:
        stats['total_words'] += 1

        if len(word) <= min_len:
            corrected = word
        elif word in word_cache:
            stats['cached_hits'] += 1
            corrected = word_cache[word]
        else:
            suggestions = sym_spell.lookup(word.lower(), Verbosity.CLOSEST, max_edit_distance=2, include_unknown=True)
            if suggestions:
                corrected = suggestions[0].term
            else:
                corrected = word
            word_cache[word] = corrected

        if corrected != word:
            stats['corrected_words'] += 1
            stats['unique_words'].add((word, corrected))

        corrected_words.append(corrected)
        
    return ' '.join(corrected_words)

In [ ]:
stats['start_time'] = time.time()

tqdm.pandas(desc='SymSpell')
df['corrected'] = df['clean'].progress_apply(spell_check_symspell) 

elapsed_time = time.time() - stats['start_time']
print_spell_check_stats(stats, elapsed_time)

SymSpell:   0%|          | 0/1000 [00:00<?, ?it/s]

SymSpell: 100%|██████████| 1000/1000 [00:00<00:00, 3174.60it/s]

Слов проверено: 89,234
Исправлено слов: 8,017 (8.98)%, уникальных слов: 1,578
Попаданий в кэш: 71,570
Скорость: 279730 слов/сек


#### PySpellChecker

In [20]:
%%script cmd /c ""
from spellchecker import SpellChecker

spell_checker = SpellChecker(language='ru')

word_cache = {}
stats = {
    'total_words': 0,
    'corrected_words': 0,
    'cached_hits': 0,
    'start_time': None,
}

def spell_check_py(text):
    if not isinstance(text, str) or not text.strip():
        return text
    
    words = text.split()
    corrected_words = []
    
    for word in words:
        if not word.strip():
            corrected_words.append(word)
            continue
        
        stats['total_words'] += 1
        
        if word in word_cache:
            stats['cached_hits'] += 1
            corrected_words.append(word_cache[word])
            continue
        
        if word not in spell_checker.unknown([word]):
            word_cache[word] = word
            corrected_words.append(word)
        else:
            corrected = list(spell_checker.correction(word))[0] if spell_checker.correction(word) else word
            
            if corrected != word:
                stats['corrected_words'] += 1
            
            word_cache[word] = corrected
            corrected_words.append(corrected)
    
    return ' '.join(corrected_words)

In [ ]:
%%script cmd /c ""
stats['start_time'] = time.time()

tqdm.pandas(desc='pyspellchecker')
df['corrected'] = df['clean'].progress_apply(spell_check_py)

elapsed_time = time.time() - stats['start_time']
print_spell_check_stats(stats, elapsed_time)

Если проверять орфографию на сыром датасете без кэша, то выйдет ~36 минут при скорости 60 слов в секунду. Примерная [таблица сравнения](https://github.com/diffitask/spell-checkers-comparison)

#### Итого исправления орфографии

Вывел сюда output, если выход ячеек сверху потеряется

```python
pyspellchecker: 100%|██████████| 1000/1000 [35:20<00:00,  2.12s/it]
Всего слов обработано: 88,529
Исправлено уникальных слов: 2,741
Попаданий в кэш: 81,487
Скорость: 42 слов/сек
```
=======================================================
```python
SymSpell: 100%|██████████| 1000/1000 [00:00<00:00, 2826.62it/s]
Слов проверено: 87,982
Исправлено слов: 8,017 (9.11)%, уникальных слов: 1,578
Попаданий в кэш: 70,319
Скорость: 245972 слов/сек
```
=======================================================
```python
Spylls: 100%|██████████| 1000/1000 [16:24<00:00,  1.02it/s]
Всего слов обработано: 88,651
Исправлено уникальных слов: 2,245
Попаданий в кэш: 81,579
Скорость: 90 слов/сек
```

Статистика по Spylls и pyspellchecker старая, так как работают 16 и 35 минут соотвественно, нету смысла повторять их из-за скорости.  
В итоге SymSpell сработал намного быстрее

### Лемматизация

In [22]:
# Сделаем все слова в именительном падеже с помощью лемматизации для лучшей обработки
# Также обработаем отдельно слова с "не" как "не_слово" так как много слово по типу "не удовлетворительно", так будет легче делать первичную классификацию
from pymorphy3 import MorphAnalyzer

morph = MorphAnalyzer()

def lemmatize_text(text):
    raw_tokens = text.split()

    lemmas = []
    for token in raw_tokens:
        token = token.strip()
        if token and token != 'не':
            normal_form = morph.parse(token)[0].normal_form
            lemmas.append(normal_form)
        else:
            lemmas.append(token)

    # Склеиваем "не" с последующими леммами
    result = []
    i = 0
    while i < len(lemmas):
        if lemmas[i] == 'не' and i + 1 < len(lemmas):
            # Соединяем лемму "не" со следующей леммой
            result.append('не_' + lemmas[i + 1])
            i += 2 # Пропускаем оба токена
        else:
            result.append(lemmas[i])
            i += 1
    
    return ' '.join(result)

In [23]:
df['lemmatized_text'] = df['corrected'].apply(lemmatize_text)

df['lemmatized_text'].iloc[5:16] # Самые короткие строки в начале датасета

5                                                                                                                                                                                                                                                                          рамка данный ип установить отношение недвижимый имущество земельный участок республика северный осетия албания р н кировский ст ца земский ул маяковский кадастровый номер здание республика северный осетия албания кировский район ст земский маяковский ул далее кадастровый номер вынести постановление запрет действие
6                                                                                                                                                                                                                                                  взыскивать денежный средство перечислять следующий реквизит получатель ооо мкк верить расчётный получатель инна ооо мкк верить кпп получатель ооо мкк верить инна кпп короче з

In [24]:
# Сохраняем для других ipynb
df.to_csv('data/processed/preprocessed.csv', index=False)